# Semantic Search with Hugging Face Embeddings

This notebook shows how to generate embeddings with Sentence Transformers and
index them in Grafito for semantic search.

Requirements (install separately):
- sentence-transformers
- faiss-cpu (optional, for FAISS backend)


In [ ]:
from __future__ import annotations

import orjson

from grafito import GrafitoDatabase

try:
    from sentence_transformers import SentenceTransformer
except Exception as exc:
    raise RuntimeError("Install sentence-transformers to run this notebook.") from exc

In [10]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

docs = [
    {"title": "Grafito Overview", "text": "Grafito is a lightweight graph database."},
    {"title": "Cypher Basics", "text": "Cypher is a declarative graph query language."},
    {"title": "Vector Search", "text": "Vector indexes power semantic search workflows."},
]
texts = [item["text"] for item in docs]
embeddings = model.encode(texts, normalize_embeddings=True).tolist()

query = "semantic search in a graph database"
query_vec = model.encode([query], normalize_embeddings=True)[0].tolist()

In [11]:
db = GrafitoDatabase(":memory:")
db.create_vector_index(
    "docs_vec",
    dim=len(embeddings[0]),
    backend="faiss",
    method="flat",
    options={"metric": "ip", "store_embeddings": True},
)

for item, vector in zip(docs, embeddings):
    node = db.create_node(labels=["Doc"], properties=item)
    db.upsert_embedding(node.id, vector, index="docs_vec")

In [12]:
# Python API search
results = db.semantic_search(query_vec, k=2, index="docs_vec", rerank=True)
[(row["node"].properties["title"], row["score"]) for row in results]

[('Cypher Basics', 0.6017134430595396),
 ('Grafito Overview', 0.585072670334845)]

In [ ]:
# Cypher procedure search (scientific notation is supported)
vector_literal = orjson.dumps(query_vec).decode('utf-8')
cypher = f"""
CALL db.vector.search('docs_vec', {vector_literal}, 2, {{labels: ['Doc'], rerank: true}})
YIELD node, score
RETURN node.title AS title, score
"""
db.execute(cypher)